In [2]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from copy import deepcopy
checkpoint = "EleutherAI/gpt-neo-2.7B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, output_hidden_states=True)

Some top models and their sizes.

**open-source**
- GPT-Neo 125 mil
- GPT-Neo 1.3 bil (same as GPT-3 Babbage)
- GPT-2 1.5 bil
- GPT-Neo 2.7 bil
- GPT-J 6 bil
- GPT-NeoX 20 bil
- Bloom: ranges from 350m to 176 bil

**closed-source**
- GPT-3: 175 bil at biggest

Link to understand GPT models better: [minGPT](https://github.com/karpathy/minGPT/blob/master/mingpt/model.py). Word embeddings are summed with positional embeddings then passed on.

Architecture for [GPT-Neo](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py).

**Test: transformer + lm_head = original model + probabilities for next tokens.**

In [3]:
# prepare inputs
raw_inputs = ["1+3=4"]
inputs = tokenizer(raw_inputs, return_tensors="pt")

# predict
outputs = model(**inputs)

# show that the lm_head is producing logits via linear transformation
trans = model._modules['transformer']
lm_head = model._modules['lm_head']
out = trans(inputs['input_ids'])
h = out['hidden_states'] # tuple of (layer x (batch_size, seq_len, hidden_size))
logits = lm_head(h[-1])  # select logits using last layer

# we got the same logits by going through the model
assert logits.shape == outputs['logits'].shape # tensor (batch_size, seq_len, vocab_size)
assert logits.sum() == outputs['logits'].sum()
assert logits.max() == outputs['logits'].max()

# naive check
print('input text:', tokenizer.decode(inputs['input_ids'][0]))

# top word embeddings
decoded_toks = tokenizer.decode(logits[0].argmax(axis=-1))
print('decoded text from hidden states', decoded_toks)

# dissect token-by-token
print('top predicted tokens at this position:')
for i, tok in enumerate(inputs.tokens()):
    print(f'{i}: ___{tok}___ --> ___{decoded_toks[i]}___')

input text: 1+3=4
decoded text from hidden states .\\4$
top predicted tokens at this position:
0: ___1___ --> ___.___
1: ___+___ --> ___\___
2: ___3___ --> ___\___
3: ___=___ --> ___4___
4: ___4___ --> ___$___


**Set up projecting back to word-vectors.**

In [141]:
# get embedding matrix
w_embed = trans.wte.weight # vocab_size, embed_dim
vocab_size = w_embed.shape[0]
embed_size = w_embed.shape[1]

# invert for unembedding
unemb_linear = nn.Linear(in_features=embed_size, out_features=vocab_size, bias=False)
pinv = torch.linalg.pinv(w_embed)
unemb_linear.weight = nn.Parameter(pinv.T)

# make sure unembedding works
ids = torch.Tensor([[16, 2, 3]]).int()
embs = trans.wte.forward(ids)
unembedded_onehot = unemb_linear(embs)
unembedded_ids = unembedded_onehot.argmax(axis=-1)
assert torch.all(unembedded_ids == ids)

**Get gradient wrt embedding vector**

In [142]:
# check that original outputs match outputs when using embedding
embeds = trans.wte.forward(ids)
outputs_using_embeds = model(inputs_embeds=embeds)
outputs = model(input_ids=ids)
assert outputs['logits'].sum() == outputs_using_embeds['logits'].sum()

# get gradient
embeds = trans.wte.forward(ids)
embeds.retain_grad()
outputs = model(inputs_embeds=embeds)
loss = outputs['logits'].sum()
loss.backward()
embeds.grad

tensor([[[  9575.5391,  -1355.6121,  -2623.6182,  ...,  12424.7637,
           -5506.7495,  -9292.2803],
         [ 12695.8623,  10005.6162,  17801.9453,  ...,   7830.0645,
           24521.3984,   4288.2812],
         [ -9870.7490,   -223.3108,  -3130.4512,  ...,  15149.9629,
          -23681.2461,  -5486.9058]]])